# Overview

The following data was created by extracting guano using the MoBio HTP Soil kit, amplifying arthropod COI fragment using new JOH primers, then 250 PE sequencing on an Illumina HiSeq2500. RunStats for these data forthcoming. 

See [UFITS Github page] **(note this was since renamed 'amptk' and has the GitHub repo was subsequently renamed too --> assume anything labeled "UFITS is referring to the renamed 'amptk' program)** [link1] for overview of install and executing code. Because program continues to be under development, use instructions on README.md file only.

This notebook will describe a modification from the traditional UFITS pipeline which uses Robert Edgar's UPARSE algorithm to merge PE reads, cluster OTUs, etc. Instead, we'll use a newly employed algorithm called DADA2 (see [paper] [link3] and [github page] [link4]) which can do the same things as the original UFITS pipeline plus error correct reads. The advantage here is we can likely keep samples which have lower numbers of reads because this algorithm has a way to error correct the Illumina reads prior to clustering the merged PE reads - however Jon Palmer continues to advise against this but in any event this algorithm seems to do a better job of calling what's there and leaving out what's not.

[link1]:https://github.com/nextgenusfs/amptk
[link3]:http://www.nature.com/articles/nmeth.3869.epdf?author_access_token=hfTtC2mxuI5t44WUhsz05NRgN0jAjWel9jnR3ZoTv0N5gu3rLNk61gF4j2hXPcagLe964qdfd3GRw8OwyZxfEgsul8lwR1lEWykR3lWF30Dl_bZWMvTPwOdrwuiBUeYa
[link4]:https://github.com/benjjneb/dada2

## Other information

- Running core programs:
    - UFITS 0.7.3
    - vsearch v2.3.4_linux_x86_64
    - usearch usearch v9.2.64_i86linux32
    - python 2.7.6
- Installed the necessary (COI) database

# part 1 - data cleaning

One of the first thing to do with the raw data from Cobb is move exactly the files you want into a specific directory. For the BRI project, there was more than just BRI samples getting sequenced, so you have to parse out the files you want from the files you don't. In this case, it's just a matter of using a wildcard with the SampleName in the header:

In [ ]:
mv bri* ./{folder-you-want}

Once that's complete, the next thing you're going to need to do is move out all the fastq.gz files out from their individual directories, again using a wildcard:

In [ ]:
mv Sample_bri16*/*.gz ./fastq_files

This will leave behind the .csv files you don't need, and keep all the sample .fastq.gz files in a single directory for UFITS to work with, as it's intended. Once that's complete, you then need to rename the files because that three-part naming schemed used to separate them out in the first place actually doesn't jive with how UFITS scripts want to parse things...It defaults to looking at that first underscore and chopping everything off after that, so samples all get merged together into one big morass, which you don't want at all.  

Instead, use the following example to rename the scripts to incorporate a hyphen, which UFITS is okay with:

In [ ]:
#note you have to be within the directory structure to run this!

rename 's/bri16_/bri16\-/' *

#this should replace only those files named "bri16_{something}..."

#you may need to rerun this command to rename files NOT named "bri16...". For example:

rename 's/negoro_/NTC\-/' *

Once everything is properly named, you can then proceed.  

This process can take a considerable amount of time; increase the number of cpus when possible is advised.  
For example, with 8 cpus on 14 samples (containing 17 Gb of data), it took ~1 hour to process. 

Data stored in various subdirectories within parent directory at: __'/leo/devon/projects/bri2016/'__.  

In [ ]:
#trim primers
ufits illumina \
-i /leo/devon/projects/bri2016/fastq_files \
-u /home/fosterLab/devonr/bin/usearch9 \
--rescue_forward on \
--require_primer off \
--min_len 160 \
--full_length \
--cpus 20 \
--read_length 250 \
-f GGTCAACAAATCATAAAGATATTGG \
-r GGWACTAATCAATTTCCAAATCC \
-o trimd

A little bit of the output will look like as follows. Double check you have observed the expected number of (barcoded) samples. Note the huge range in number of reads passing filter per sample (more on that below).

In [ ]:
#-------------------------------------------------------
#[12:39:03 PM]: Concatenating Demuxed Files
#[12:39:10 PM]: Counting FASTQ Records
#[12:39:13 PM]: 6,496,299 reads processed
#[12:39:25 PM]: Found 351 barcoded samples
#                        Sample:  Count
#                    bri16-0091:  147836
#                    bri16-0269:  139179
#                    ...
#                    bri16-0174:  1478
#                    NTC-28C03:  1373
#                    bri16-0401:  1372
#                    ...
#                    bri16-0370:  36
#                    bri16-0372:  33
#                    bri16-0349:  32
#[12:39:25 PM]: Output file:  trimd.demux.fq (2.4 GB)
#[12:39:25 PM]: Mapping file: trimd.mapping_file.txt
#-------------------------------------------------------

Output from this script will contain several new files:  

1. In the directory in whch the script was executed:
    - (output_name).mergedPE.log containing information about PE merging. Note that information in this single file includes the summary of all individual merged pairs.
    - (output_name)-filenames.txt containing information about files used in this pipeline (such as index-pair combinations)
    - (output_name).demux.fq containing a concatenated file of all trimmed and PE merged sequences of all samples listed in the '-filenames.txt' file above __(this one is to be used in the subsequent OTU clustering step)__

2. In the (output_name) directory named in the 'ufits illumina ...' argument provided above:  
    - (output_name).ufits-process.log containing information about the demultiplexing and read trimming processes
    - a single sample_name.fq file which is the PE merged file from each sample_name...R1/R2 pair of raw fastq inputs
    - a single sample_name.demux.fq file for each PE merged sample_name.fq file having been trimmed as defined  

**ufits-process.log** will also contain a summary of information regarding the total number of reads processed as well as the number of reads processed per sample. Note in the little bit of sample output above how there's about huge range in reads per sample - from a high of about 150,000 reads all the way down to less than 30. We're going to want to set a minimum number of reads required for a given guano sample to be kept for future analyses, otherwise we'll run into problems with index-bleed calculations.

In [ ]:
#                    Sample:  Count
#                    bri16-0091:  147836
#                    bri16-0269:  139179
#                    ...
#                    bri16-0174:  1478
#                    NTC-28C03:  1373
#                    bri16-0401:  1372
#                    ...
#                    bri16-0370:  36
#                    bri16-0372:  33
#                    bri16-0349:  32

If at this point it's valuable to point out that you may want to exclude certain samples which contain too few reads. These can be discarded at this point before moving forward in the OTU clustering part. Jon Palmer recommends shooting for samples with about 50,000 reads, though as low as 10,000 samples are fine. In the current project we're going to remove samples with less than 5000 reads that pass our filters. While this is below the 10,000 value Jon recommends, it's nearly rx higher than the first NTC to show any reads (at 1373).  

Use the following command to *remove* the samples that are below that 5000-read minimum threshold:

In [ ]:
ufits remove \
-i {name.of.demux.file} \
-f samples2drop.txt \
-o {name.of.output.fq}

#for example, with our big dataset, we could use:
ufits remove \
-i trimd.demux.fq \
-f samples2drop.txt \
-o cleaned.demux.fq

- the -i file should be something named like "samples.demux.fq"  
- the -o file would be what you're cleaned up demux'd .fq file will be called
- the -f file should be a single column text file with the names of each file

You'll generate a new {output.name}.fq file containing only the samples you wanted to keep (or, without all the samples you just dropped). In addition, following the **ufits remove** command you'll see an output with some basic information about how many samples were removed and how many reads were kept:

In [ ]:
#Removed 171 samples
#Kept 6377119 reads out of 6529250 total reads

If you want to double check that new .fq file contains just what you want, run the following command to double check you didn't miss anything:

In [ ]:
ufits show -i {new.output.fq}

This should produce a list of the barcoded samples that remain; confirm you have the samples you want and don't have samples you intended on discarding. Once that's done it's on to OTU clustering.

# part 2 - OTU clustering

Two different clustering options to choose from here: UPARSE or DADA2. Both of the following scripts incorporate a chimera filtering step with UCHIME.

### Side note - before you cluster!

If you happen to have multiple mock communities, it's advised to not proceed with the clustering just yet. Unfortunately it's not obvious to indicate what these problems will be unless you continue with clustering *without* merging the mock community sample into a single larger sample, but suffice it to say I've explored the effects of working with multiple mock communities and it creates some problems.  

In this current project, I used two mock community spike-ins: they were identical in their community representation and can be treated as sequencing duplicates (they're just PCR replicates from the same DNA tube); these are listed as **mock-8000A** and **mock-8000B**.  Unfortunately, this wasn't advised by Jon P. and he advocated an approach of merging the mock communities instead of using them as indepdendent samples. Merging the two mock communities is effectively averaging out the effect of the index-bleeds which are acquired independently; on one hand this is nice as it smooths out the effects that may be pronounced in one mock sample and reduced in another, and given that index-bleed is partly sequence specific (we think?), averaging may not be the perfect route. On the other hand, having multiple mocks (and using them as distinct samples) may allow you to avoid being overly harsh on any one OTU given that you can see the per-OTU per-mock sample variation in index bleed.  

My current solution is to just merge these two mock communities into a single mock sample, and proceed with clustering and filtering using that merged collection of reads. To do that, we're going to just rename one of the two mocks as the other, then relabel all of them once more as a single mock community:  

First, rename **mock-8000A** as **mock-8000B*

In [ ]:
sed -i 's/barcodelabel=mock-8000A;/barcodelabel=mock-8000B;/g' cleaned.demux.fq

Then rename all the samples now labeled as **mock-8000B** as just **mockIM3** to remind yourself that these are from the IM3 set of mock community samples

In [ ]:
sed -i 's/barcodelabel=mock-8000B;/barcodelabel=mockIM3;/g' cleaned.demux.fq

In [ ]:
#for UPARSE clustering
ufits cluster \
--fastq cleaned.demux.fq \
--uchime_ref COI
-o UPout

#for DADA2 clustering
ufits dada2 \
--fastq cleaned.demux.fq \
--uchime_ref COI \
-o D2clust \
--length 180 \
--platform illumina

This will produce the following output files
   - **{out.name}.cluster.otus.fa** (use this fasta file in the next command)
   - **{out.name}.otu_table.txt** (use this fasta file in the next command)
   - **{out.name}.ufits-cluster.log**  

Here's a sample output using the default settings with UPARSE:

In [ ]:
# -------------------------------------------------------
# [01:49:15 PM]: OS: linux2, 32 cores, ~ 264 GB RAM. Python: 2.7.6
# [01:49:15 PM]: ufits v.0.7.3, USEARCH v9.2.64, VSEARCH v2.3.4
# [01:49:15 PM]: Loading FASTQ Records
# [01:49:18 PM]: 6,377,119 reads (2.4 GB)
# [01:49:18 PM]: Quality Filtering, expected errors < 1.0
# [01:53:48 PM]: 6,308,755 reads passed
# [01:53:48 PM]: De-replication (remove duplicate reads)
# [01:54:14 PM]: 427,595 reads passed
# [01:54:16 PM]: Clustering OTUs (UPARSE)
# [01:54:37 PM]: 2,365 OTUs
# [01:54:37 PM]: Cleaning up padding from OTUs
# [01:54:37 PM]: Chimera Filtering (VSEARCH) using COI DB
# [01:54:43 PM]: 1,690 OTUs passed, 675 ref chimeras
# [01:54:43 PM]: Mapping Reads to OTUs and Building OTU table
# [01:58:28 PM]: 6,266,699 reads mapped to OTUs (98%)
# -------------------------------------------------------

And here's the output using DADA2. Note that there are the iSeqs clustered (which involves DADA2 itself), but what we're ultimately doing is taking all the reads that pass our DADA2 filters then passing the DADA.fa file with our reads to generate clusters using VSEARCH.  

The important piece to observe is the final OTUs clustered at 97% identity.

In [ ]:
# -------------------------------------------------------
# [08:34:15 AM]: OS: linux2, 32 cores, ~ 264 GB RAM. Python: 2.7.6
# [08:34:15 AM]: ufits v.0.7.3, USEARCH v9.2.64, VSEARCH v2.3.4
# [08:34:15 AM]: Loading FASTQ Records
# [08:34:19 AM]: 6,377,119 reads (2.4 GB)
# [08:35:02 AM]: Quality Filtering, expected errors < 1.0
# [08:37:24 AM]: 6,307,608 reads passed
# [08:37:54 AM]: DADA2 compatible read lengths, avg: 181 bp, min: 160 bp, max: 310 bp, top 95%: 181 bp
# [08:37:54 AM]: Splitting FASTQ file by Sample and truncating to 180 bp
# [08:41:25 AM]: Running DADA2 pipeline
# [09:08:35 AM]: R v3.3.2, DADA2 v1.3.0
# [09:08:35 AM]: 5,242 total inferred sequences (iSeqs)
# [09:08:35 AM]: 3,369 denovo chimeras removed
# [09:08:35 AM]: 1,873 valid iSeqs
# [09:08:35 AM]: Chimera Filtering (VSEARCH) using COI DB
# [09:08:43 AM]: 1,799 iSeqs passed, 74 ref chimeras removed
# [09:08:44 AM]: Mapping reads to DADA2 iSeqs
# [09:16:12 AM]: 6,241,962 reads mapped to iSeqs (98%)
# [09:16:12 AM]: Clustering iSeqs at 97% to generate biological OTUs
# [09:16:13 AM]: 1,197 OTUs generated
# [09:16:13 AM]: Mapping reads to OTUs
# [09:21:14 AM]: 6,217,520 reads mapped to OTUs (97%)
# -------------------------------------------------------

**Big difference in OTUs** between the UPARSE and DADA2 filtering approaches - we get about 400 *less* OTUs with DADA2, which is probably the better option.  I've subsequently compared these two clustered outputs using each of the commands listed below and it appears that the DADA2 algorithm is the one that produces the cleanest data (fewest OTU contaminants in mock community). It's the more conservative approach which is what we're going for in this preliminary work.

# Part 3: filtering OTU table

It's advocated by Jon Palmer to use an index-bleed filter command using the '-b mock' flag, specifying the use of a mock community to calculate the index-bleed percentage. The idea behind this filtering step is to identify the number of instances in which a read mapped to an OTU which is *not supposed to be in the mock community* is found in the mock community. The percentage represents the overall number of reads that map to mock OTUs (ie. that are supposed to be in there) relative to the number of reads from OTUs that aren't. The alternative approach is to just trim down reads by some defined (yet arbitrary) percent across all samples, given some other examples in other data sets. I don't like that because I've found that every data set is unique, so having a mock community to judge this by is better.  

We're going to use our mock community (albeit a merged pair of communities... see the "side note" in part 2 above) to calculate index bleed. We're also going to (potentially) incorporate a subtraction value in which *if* the scenario occurs such that an unwanted OTU remains in the mock community following the application of the index-bleed filter, we can detect how many reads that maximum value should be and subtract from there to ensure that **zero** normalized reads remain in the highest 'bleeding' OTU in the mock sample. This value is subtracted from *all* reads from *all* samples per OTU, not just the mock community, so it ultimately drops a lot of OTUs with low read thresholds.  

There are intermediate files which are very useful in determing exactly how many read of which OTU are creeping into your mock community (that are unwanted OTUs), so you'll notice we're passing a **"--debug"** flag which is used to generate these files; without that flag, you'll miss the normalized read counts used in calculating the index bleed filter as well as the subsequent "--subtract" value.

You might want to play with the **--subtract** threshold and examine how many overall OTUs are retained. In the two examples below, we're going to play with the most conservative approach (filt1) versus a slightly less harsh filter (filt2).  
- **filt1** uses two flags: the index-bleed flag, and the **--subtract auto** flag. This second flag will calculate what the total number of normalized reads are in OTUs unwated in the mock community and then subtract that value from all reads. This results a completely clean mock, but drops a lot of samples
- **filt2** uses the same two flags, but lowers subtraction value used in **filt1**. I'm basing this lower value by viewing the file **.normalized.num.txt**, and moving to the second most abundant read to determine what that subtractoin value was

First example allows for an automatic detection and application of index-bleed and OTU subtraction (if necessary):

In [ ]:
ufits filter \
-i D2clust.cluster.otu_table.txt \
-f D2clust.cluster.otus.fa \
-b mockIM3 \
--keep_mock \
--mc /leo/devon/projects/nhguano2016/CFMR_insect_mock3.fa \
--debug \
--subtract auto \
-o filt1

The resulting output will show you that:
- there was an index bleed of 1.6% applied (that's pretty high!), and 
- the auto subtract filter was used at a level of 204.  

In other words after filtering down all reads by 1.6% on a per-OTU basis, all reads were then subtracted form each sample's OTU by an additional 204 reads. That's a lot. In fact, it drops the total number of OTUs present in the table from **1197 OTUs** down to **515 OTUs** (more than half!).  

The next consideration is whether or not the OTUs could be kept if we were imposing a less stringent standard. If you run the same command (above) without passing the **--subtract auto** flag, you'll keep nearly all original OTUs. This indicates that it's this **--subtract** flag that's dropping most of our information.  

To investigate what's going on we're going check out the **{name}.normalized.num.txt** file and compare it with the **.final.txt** file. We need to figure out which OTUs in the mock community have the greater number of OTUs that are *unwanted* - that is, OTUs which should not be in the mock community. We're going to clean up the output of the needed files just a bit so that our search basic linux tools work properly (this is because the space in the first line of the files shifts the first row into one more field than all the rows below).

Run the following commands:

In [ ]:
sed -i 's/#OTU ID/OTUid/' filt1.final.txt
sed -i 's/#OTU ID/OTUid/' filt1.normalized.num.txt

You don't know two things at this point: 
- how many fields are there?
- which OTUs contain the most reads among OTUs that **don't** belong?  

To answer the *number of fields question*, substitute **file.name.txt** with whatever file you want:

In [ ]:
awk -F '\t' '{print NF; exit}' file.name.txt

For example:

In [ ]:
awk -F '\t' '{print NF; exit}' filt1.final.txt

Returns the value:

In [ ]:
# 182

So there are 182 fields. that's important when you want to next figure out which columns to sort. Fortunately with our current data set the last field is the mock community, and the first field is the list of OTUs in each sample. So we're going to just sort through this big array of a text file and just print out the three fields we want.  The following command should give you a sense of how many reads may need to be subtracted from a given OTU that's unwanted.

In [ ]:
#for the .normalized.num.txt file:
cut filt1.normalized.num.txt -f 1,182 | sort -k2,2n | awk '$2 != "0.0" {print $0}' 

#for the .final.txt file:
cut filt1.final.txt -f 1,182 | sort -k2,2n | awk '$2 != "0" {print $0}'

You should see lists of two columns of data representing the number of reads present for each OTU *for OTUs in which there are > 0 reads*. In the **filt1.normalized.num.txt** file there are many OTUs besides those identified as mock community members (add on with an extra pipe to the above command for the **.normalized.num.txt** file to get a total count: *grep -c -v "MockIM"*). 

What should become obvious is that the majority of these OTUs could be filtered out with a lower **--subtract** value than the automatic one of **204** which would lead to keeping more OTUs in our dataset. It's why there are only OTUs that should be present in that **final.txt** file output.  

To confirm that this is really a result of index-bleed and not a result of chimera misidentification, look at each of the the top *unwanted* OTUs from that **normalized.num.txt** file individually with the following grep command and confirm that what you're seeing is really index-bleed.  

For example, to see how many reads are in every sample of the data set for **OTU55** (the highest read number of any *unwanted OTU* in our mock community, run:

In [ ]:
grep "\\bOTU55\\b" filt1.normalized.num.txt

Which will result in the following output (some columns have been truncated for sake of clairty):

Notice how just a few samples contain the vast majority of the reads. This is a clear indication of index-bleed, as chimeric sequences would typically how low read abundance in many samples (the opposite of what we see above).  

You can run the same *grep* command for each OTU of concern and you'll notice that each case gives the same impression of likely index-bleed.  

We're going to therefore apply a **--subtract** filter using a value that will retain the three highest *unwanted OTUs* while cutting out all the others; this is a moderate approach at reducing the unwanted effect of index bleed without losing to much data. If you look back at the earlier output from the **cut filt1.normalized.num.txt -f 1,182...** command, you'll notice that a value of **86** would remove all but the top 3 of the unwanted OTUs, which does a good job of balancing keeping as much data as possible while also further reducing potential effects of index bleed. It's not perfect, but it's better than nothing, and it's an empirically derived value.

The following command creates the output for the final set of files we will use to then assign taxonomy. A few things of note:
- the change in the **--subtract** argument from *auto* to *86*.
- we've dropped the **--keep_mock** flag because we're no long going to need those data for taxonomy assignment
- we've added the **--delimiter tsv** arugment to produce tab-delimited text file as our output

In [ ]:
ufits filter \
-i D2clust.cluster.otu_table.txt \
-f D2clust.cluster.otus.fa \
-b mockIM3 \
--mc /leo/devon/projects/nhguano2016/CFMR_insect_mock3.fa \
--subtract 86 \
--delimiter tsv \
-o filtd

And the output:

In [ ]:
# -------------------------------------------------------
# [05:50:11 PM]: OS: linux2, 32 cores, ~ 264 GB RAM. Python: 2.7.6
# [05:50:11 PM]: ufits v.0.7.3, USEARCH v9.2.64, VSEARCH v2.3.4
# [05:50:11 PM]: Loading OTU table: D2clust.cluster.otu_table.txt
# [05:50:11 PM]: OTU table contains 1197 OTUs
# [05:50:11 PM]: Mapping OTUs to Mock Community (USEARCH)
# [05:50:12 PM]: Sorting OTU table naturally
# [05:50:12 PM]: Removing OTUs according to --min_reads_otu: (OTUs with less than 2 reads from all samples)
# [05:50:12 PM]: Normalizing OTU table to number of reads per sample
# [05:50:12 PM]: Index bleed, samples into mock: 1.528107%.
# [05:50:12 PM]: Will use value of 1.600000% for index-bleed OTU filtering.
# [05:50:12 PM]: Subtracting 86 from OTU table
# [05:50:12 PM]: Filtering OTU table down to 685 OTUs
# [05:50:12 PM]: Filtering valid OTUs
# -------------------------------------------------------

Filtering choices make a big difference!! We go from **1197 OTUs** after our DADA2 pipeline, then our initial filtering attempts chopped that all the way down to **515 OTUs**. The final, slightly more moderate attempt ultimately retained **685 OTUs**.  

The arguments above will produce the following files:  
- **out.filtered.otus.fa** is the final filtered fasta file (say that five times fast)
- **out.final.binary.csv** is the presence/absence OTU table after filtering
- **out.stats.csv** is the number of OTUs in each sample before and after filtering
- **out.final.csv** is the OTU table with normalized read counts (noramlized to the number of reads in each sample)
- **out.ufits-filter.log**  

The next step is to assign taxonomy by using the **out.filtered.otus.fa** and **out.final.binary.csv** files. 

## Part 4: Assigning Taxonomy to OTUs

Make sure to have already downloaded the necessary database (COI). Need to run this command for every instance in which you've generated a uniquely filtered dataset from Parts 1-3. Here's just a pair of examples you'd run in their respective directories:

In [ ]:
ufits taxonomy -i filtd.final.binary.txt -f filtd.filtered.otus.fa -d COI -o D2bri-chunk1

Which produces the following summary (note the OTU classification is performed with UTAX, using a database acquired through BOLD; see J. Palmer for details about its creation).

## What do we find?

See the R script 'ufits_2016_OTUtable_analysis.R' for generating the following observations.  

Move the files as follows:

In [ ]:
#note the files will be copied to the current directory:
rsync devon@pinky.sr.unh.edu:/leo/devon/projects/bri2016/bri-chunk1/D2bri-chunk1* .

# BLASTing some of those unknowns 

We can use the following scripts to run NCBI BLAST. This will happen in three steps:

- A. Clean up the file to pull out just the UTAX-tagged OTUs and convert into a single-line fasta
- B. Build a blast database (* note we're not going to use this step at the moment*)
- C. BLAST the file

**A.** First, use fastx toolkit to convert '{name}.otus.taxonomy.fa' into oneliner fasta

In [ ]:
fasta_formatter -i D2bri-chunk1.otus.taxonomy.fa -o allOTUs_oneliner.fasta -w 0

Then grep only lines with "UTAX" or "SINTAX" in header and remove the dashes where the lines were removed

In [ ]:
egrep '(UTAX|SINTAX)' allOTUs_oneliner.fasta -A 1 | sed '/^-/d' > blastit.fa

**B.** We're going to skip the step of setting up a BLAST database for the moment - there's already the complete 'nr' database installed on Pinky, so unless we want to set up a custom database with specific sequences we don't have available through NCBI, then I wouldn't worry about this step. If you did want to do it, you'd run the following command:

In [ ]:
makeblastdb -in {blast_subjects.fasta} -parse_seqids -dbtype nucl -out {path.to}/my_db

**C.** Finally, we're going to run a BLAST search on the UTAX and SINTAX sequences using NCBI's nr database. We're going to specify a few other flags described in the code below. Specifically, we're going to take only values that are a certain alignment length, a certain percent identity, and then filter these to choose the best possible taxonomic description to those collapsed reads.  

See [here] [link1] for BLAST manual with command line options used below.
[link1]:http://www.ncbi.nlm.nih.gov/books/NBK279675/

In [ ]:
blastn \
-query /leo/devon/projects/bri2016/bri-chunk1/blast_out/blastit.fa \
-db /opt/ncbi-blast-2.2.29+/db/nt \
-outfmt '6 qseqid sseqid pident length bitscore evalue staxids' \
-num_threads 8 \
-perc_identity 79.9 \
-max_target_seqs 12 \
-out /leo/devon/projects/bri2016/bri-chunk1/blast_out/blastout.txt

To get the taxonomy information from these taxids we're going to use an R package called [taxize] [link1] to convert the *staxids* value into taxonomic information (kingdom, phylum,...species).  The only information we need to provide to **taxize** is a list of the taxids.  

However, we're going to sort through our blast output a little bit first.

[link1]:https://github.com/ropensci/taxize

#1. because I had included a blast result column with taxonmy names that didn't work and printed "N/A" I removed it and created tab-separated fields as follows:

In [ ]:
awk '{print $1,'\t',$2,'\t',$3,'\t',$4,'\t',$5,'\t',$6,'\t',$7}' blastout.txt > clnblastout.txt

#2. I then removed redundant rows that contained identical OTU id's, bit scores, and taxIDs. I also then removed any rows in which the alignment length of a match was less than 150 base pairs:

In [ ]:
sort -u -k1,1 -k5,5nr -k7,7 clnblastout.txt | awk '$4 > 149 {print $0}' > cln2blastout.txt

#3. I then filtered out any redundant row with common OTUid and TAXid, then sorted by bit score (highest first).

In [ ]:
sort -u -k1,1 -k7,7 cln2blastout.txt | sort -k1,1 -k5,5nr > cln3blastout.txt

This results in filtering down from the initial **2295 rows** to **903 rows**. We'll use these results to investigate whether or not the taxonomy information assigned by UTAX/SINTAX could be updated or not.  

The next step is to import these TAXid values into R and use a package called **taxize**.  This information is available on my [GitHub repo] [link1].  

You can compare what you're trying to clean up with what's already been assigned by the original SINTAX or UTAX classifcation in the .fa file from the UFITS taxonomy output. You're going to need to get it into a format that R won't hate you for though:

[link1]:https://github.com/devonorourke/guano/blob/master/R_scripts/taxifying_blastout.R

In [ ]:
awk 'BEGIN{FS=" |;|,"; OFS = "\t"}{print $1, $2, $3, $4, $5, $6, $7, $8, $9}' taxheadsonly.txt > clntaxheads.txt

check out this link for details about why we're filtering how we are... https://edwards.sdsu.edu/research/percent-similarity-at-different-taxonomic-levels/'  

The next step is to move on to a bit of manual curation with the output of the Taxize script.

# Taxize list to manually update

The output of the work in the Taxize R script above is to improve the taxonomic classification of the reads assigned to some OTU from the SINTAX and UTAX algorithms during 'ufits taxonomy'.  We're going to use the updated information from the R script to manually alter these classifications.  

To generate the file to modify in Excel:

In [ ]:
cut -f 1,182 D2bri-chunk1.otu_table.taxonomy.txt > OTUs_tomodify.txt

You'll open up this file in Excel and split into respective fields. Then concatenate into the necessary format following the syntax of the fasta file headers in the UFITS output.  

For example, all headers start with a sequence identifier and taxID of some sort, followed by a semi colon, followed by "tax=:", then the abbreviated taxonomic level followed by a colon, followed by that taxonomic level description, followed by a comma, etc. until you get to the species. Note that if incomplete information is used, then you just leave those sections blank.  

Here's a complete one:
**BOLD:AAA9568;tax=k:Animalia,p:Arthropoda,c:Insecta,o:Coleoptera,f:Cerambycidae,g:Monochamus,s:Monochamus scutellatus**  

And here's an incomplete one:
**SINTAX;k:tax=Animalia,p:Arthropoda,c:Insecta**

Once you've updated the names as appropriate, you're going to assign the updated names to the same OTUs as follows. Use the file of the fixed OTUs from Excel and incorporate as follows (it's the file called **"excel_blastUpdated.txt"**)

In [ ]:
#create file with OTUs to search for ==> "searchfile.txt"
## OTU100
## OTU1034
## OTU113
## ... 43 more ...
## OTU934

#search in one-line fasta for these terms, and get rid of hyphen separating returned values:
grep -Fwf ../excel_blastUpdated.txt allOTUs_oneliner.fasta | sed '/^-/d' > fastatoFix.fa


#convert fasta file to tab-delimited text file, and get just the second column of this file (the sequences):
awk 'BEGIN{RS=">"}NR>1{sub("\n","\t"); gsub("\n",""); print RS$0}' fasta_toFix.fa | cut -f 2 > paste2.txt


#create file with replacement classification scheme, created using blast output and excel, concatenating taxonomic information...
>ncbi:258930;tax=k:Animalia,p:Chordata,c:Mammalia,o:Chiroptera,f:Vespertilionidae,g:Lasiurus,s:Lasiurus borealis
>ncbi:1822957;tax=k:Animalia,p:Arthropoda,c:Insecta,o:Diptera,f:Empididae
>ncbi:1770991;tax=k:Animalia,p:Arthropoda,c:Insecta,o:Coleoptera,f:Elateridae,g:Melanotus,s:Melanotus similis
... 43 more ...
>ncbi:1847630;tax=k:Animalia,p:Arthropoda,c:Insecta,o:Lepidoptera,f:Tortricidae,g:Phiaris,s:Phiaris bipunctana


#paste the replacement taxonomic information with the sequences:
paste paste1.txt paste2.txt > converttofasta.txt


#create a new fasta file from that tab-delimited text file:
awk -F '\t' '{print $1"\n"$2}'  converttofasta.txt > forfastX.fa

#convert back to 60-character multiline fasta using fastx toolkit's "FASTA Formatter"
fasta_formatter -i forfastX.fa -w 60 > addthese.fa

#verify they look correct:
head -n 5 addthese.fa 
    # returns
## >ncbi:258930;tax=k:Animalia,p:Chordata,c:Mammalia,o:Chiroptera,f:Vespertilionidae,g:Lasiurus,s:Lasiurus borealis
## CACTTTATATTTACTATTTGGCGCTTGAGCGGGAATAGTTGGGACTGCATTAAGCCTATT
## AATTCGAGCTGAATTAGGCCAACCAGGTGCTCTTTTAGGAGATGATCAAATCTATAACGT
## TATCGTGACTGCCCATGCATTCGTAATAATTTTTTTTATAGTCATGCCCATTATAATTGG
## >ncbi:1822957;tax=k:Animalia,p:Arthropoda,c:Insecta,o:Diptera,f:Empididae
## AACTCTCTATTTCATCTTTGGAGCATGAGCAGGAATGCTAGGAACATCTCTAAGACTCTT    

## then!!! create a new ufits DB (or rather, update what we have)

#step 1 - get length of original COI.extracted.fa file:
grep -c "^>" COI.extracted.fa
## returns value of 332329

#step 2 - create a new .fa file by adding these additional fasta sequences:
    #perform in .../ufits-v0.7.2/DB/ directory
    cat COI.extracted.fa addthese.fa > COI.updated.fa
    #confirm it's added the right number of sequences:
    grep -c "^>" COI.updated.fa
        #returns value of 332376 - good! it's 47 longer than the original, and that's how many new records we added

#step 3 - update the database in ufits with this new fasta
ufits database -i COI.updated.fa -o COI --create_db usearch --keep_all --skip_trimming --format off

# Round 2 - redoing the analyses with the updated DB

See the extensive notes above for explanations about how and why each of the following commands are executed. 

## Cluster again

Start at Part 2 - clustering with DADA2

In [ ]:
#running in: /leo/devon/projects/bri2016/bri-chunk1/updatedDB

ufits dada2 \
--fastq /leo/devon/projects/bri2016/bri-chunk1/cleaned.demux.fq \
--uchime_ref COI \
-o clustR2 \
--length 180 \
--platform illumina

Output differs slightly from before - more chimeras removed; otherwise pretty much identical output.

In [ ]:
## -------------------------------------------------------
## [01:05:39 PM]: OS: linux2, 32 cores, ~ 264 GB RAM. Python: 2.7.6
## [01:05:39 PM]: ufits v.0.7.3, USEARCH v9.2.64, VSEARCH v2.3.4
## [01:05:39 PM]: Loading FASTQ Records
## [01:05:42 PM]: 6,377,119 reads (2.4 GB)
## [01:06:25 PM]: Quality Filtering, expected errors < 1.0
## [01:08:41 PM]: 6,307,608 reads passed
## [01:09:09 PM]: DADA2 compatible read lengths, avg: 181 bp, min: 160 bp, max: 310 bp, top 95%: 181 bp
## [01:09:09 PM]: Splitting FASTQ file by Sample and truncating to 180 bp
## [01:12:40 PM]: Running DADA2 pipeline
## [01:35:10 PM]: R v3.3.2, DADA2 v1.3.0
## [01:35:10 PM]: 5,242 total inferred sequences (iSeqs)
## [01:35:10 PM]: 3,369 denovo chimeras removed
## [01:35:10 PM]: 1,873 valid iSeqs
## [01:35:10 PM]: Chimera Filtering (VSEARCH) using COI DB
## [01:35:17 PM]: 1,784 iSeqs passed, 89 ref chimeras removed
## [01:35:18 PM]: Mapping reads to DADA2 iSeqs
## [01:43:03 PM]: 6,213,934 reads mapped to iSeqs (97%)
## [01:43:03 PM]: Clustering iSeqs at 97% to generate biological OTUs
## [01:43:04 PM]: 1,183 OTUs generated
## [01:43:04 PM]: Mapping reads to OTUs
## [01:48:32 PM]: 6,172,191 reads mapped to OTUs (97%)
## -------------------------------------------------------

## Filter again

Determine if OTUs in mock look similar and if similar **index-bleed** and **substract** values are warranted.

In [ ]:
#running in: /leo/devon/projects/bri2016/bri-chunk1/updatedDB

ufits filter \
-i clustR2.cluster.otu_table.txt \
-f clustR2.cluster.otus.fa \
-b mockIM3 \
--keep_mock \
--mc /leo/devon/projects/nhguano2016/CFMR_insect_mock3.fa \
--debug \
--subtract auto \
-o filt1

Output looks identical to previous run...

In [ ]:
## -------------------------------------------------------
## [02:01:25 PM]: OS: linux2, 32 cores, ~ 264 GB RAM. Python: 2.7.6
## [02:01:26 PM]: ufits v.0.7.3, USEARCH v9.2.64, VSEARCH v2.3.4
## [02:01:26 PM]: Loading OTU table: clustR2.cluster.otu_table.txt
## [02:01:26 PM]: OTU table contains 1183 OTUs
## [02:01:26 PM]: Mapping OTUs to Mock Community (USEARCH)
## [02:01:26 PM]: Sorting OTU table naturally
## [02:01:26 PM]: Removing OTUs according to --min_reads_otu: (OTUs with less than 2 reads from all samples)
## [02:01:26 PM]: Normalizing OTU table to number of reads per sample
## [02:01:26 PM]: Index bleed, samples into mock: 1.531092%.
## [02:01:26 PM]: Will use value of 1.600000% for index-bleed OTU filtering.
## [02:01:26 PM]: Auto subtract filter set to 204
## [02:01:26 PM]: Subtracting 204 from OTU table
## [02:01:27 PM]: Filtering OTU table down to 513 OTUs
## [02:01:27 PM]: Filtering valid OTUs
## -------------------------------------------------------

Evaluate these files:

In [ ]:
#cleanup
sed -i 's/#OTU ID/OTUid/' filt1.final.txt
sed -i 's/#OTU ID/OTUid/' filt1.normalized.num.txt

#how many fields
awk -F '\t' '{print NF; exit}' filt1.final.txt
    #confirm the last field is the mock community
        #yep.

where are the problems?

In [ ]:
#for the .normalized.num.txt file:
cut filt1.normalized.num.txt -f 1,182 | sort -k2,2n | awk '$2 != "0.0" {print $0}'

shows the exact same number of normalized counts as the earlier data set. The third OTU listed here was further analyzed with an NCBI BLAST search from the **cluster.otus.fa** file. This confirms that OTU55 from our first clustering in the previous data set is the same sequence as OTU54 here.  

In other words, renaming these new sequences to our database didn't change anything about the mock community (as expected!).

In [ ]:
## OTU8	108.0
## OTU82	173.0
## OTU54	204.0
## MockIM34_pident=100.0_OTU386	445.0
## MockIM49_pident=100.0_OTU219	2734.0
## MockIM15_pident=99.4_OTU189	4958.0
## MockIM6_pident=100.0_OTU190	5001.0
## MockIM40_pident=100.0_OTU167	7614.0
## MockIM46_pident=100.0_OTU161	9576.0
## MockIM39_pident=100.0_OTU157	10076.0
## MockIM41_pident=97.2_OTU155	10746.0
## MockIM5_pident=100.0_OTU141	12702.0
## MockIM4_pident=100.0_OTU127	15324.0
## MockIM21_pident=97.8_OTU154	19287.0

We're going to apply the exact same filtering parameters as before, given our mock community looks identical:

In [ ]:
ufits filter \
-i clustR2.cluster.otu_table.txt \
-f clustR2.cluster.otus.fa \
-b mockIM3 \
--mc /leo/devon/projects/nhguano2016/CFMR_insect_mock3.fa \
--subtract 86 \
--delimiter tsv \
-o filtd

And we get identical outputs as before:

In [ ]:
## -------------------------------------------------------
## [02:10:50 PM]: OS: linux2, 32 cores, ~ 264 GB RAM. Python: 2.7.6
## [02:10:50 PM]: ufits v.0.7.3, USEARCH v9.2.64, VSEARCH v2.3.4
## [02:10:50 PM]: Loading OTU table: clustR2.cluster.otu_table.txt
## [02:10:50 PM]: OTU table contains 1183 OTUs
## [02:10:50 PM]: Mapping OTUs to Mock Community (USEARCH)
## [02:10:50 PM]: Sorting OTU table naturally
## [02:10:50 PM]: Removing OTUs according to --min_reads_otu: (OTUs with less than 2 reads from all samples)
## [02:10:50 PM]: Normalizing OTU table to number of reads per sample
## [02:10:50 PM]: Index bleed, samples into mock: 1.531092%.
## [02:10:50 PM]: Will use value of 1.600000% for index-bleed OTU filtering.
## [02:10:51 PM]: Subtracting 86 from OTU table
## [02:10:51 PM]: Filtering OTU table down to 685 OTUs
## [02:10:51 PM]: Filtering valid OTUs
## -------------------------------------------------------

Then assign taxonomy.

In [ ]:
ufits taxonomy \
-i filtd.final.binary.txt \
-f filtd.filtered.otus.fa \
-d COI \
-o hybrid

#note you can also pass the following command to compare output with COI database-only calls
ufits taxonomy \
-i filtd.final.binary.txt \
-f filtd.filtered.otus.fa \
-d COI \
-o COIonly \
--method usearch